In [1]:
import sqlite3
import re
import csv

In [ ]:
def create_table(table_name, pr_base=None, pr_table=None, pr_file=None):
    cur.execute('''CREATE TABLE IF NOT EXISTS {table_name}
    (ID INTEGER, ADDRESS text, KOR text, TRANSLIT text, BASE text, SUF text, GLOSS text, AUTO_GLOSS text, TRANS text, ContKOR text, RUS text, WClass text, COMMENT text, PRIMARY KEY (ID  AUTOINCREMENT))'''.format(table_name=table_name))
    data = None
    if pr_base is not None:
        con2 = sqlite3.connect(pr_base)
        cur2 = con2.cursor()
        cur2.execute('''SELECT ADDRESS, KOR, ОСНОВА, ХВОСТ, GLOSS, TRANS, ContKOR, RUS, WClass, Поле1 FROM {pr_table}'''.format(pr_table=pr_table))
        data = cur2.fetchall()
        con2.commit()
        con2.close()
    if pr_file is not None:
        data = []
        with open(pr_file) as f:
            if pr_file[-3:] == 'tsv':
                dl = '\t'
            else:
                dl = ','
            for row in csv.reader(f, delimiter=dl):
                if pr_file not in ['Nastya2.tsv', 'Nastya3.tsv']:
                    data.append(row[1:])
                else:
                    data.append(row)
        data = data[1:]
        if 'Denis' in pr_file or 'Nastya' in pr_file:
            data = [rw[:2] + rw[3:7] + rw[8:6:-1] + rw[2:3] + rw[-1:] for rw in data]
        elif 'Seva' in pr_file:
            data = [rw[:2] + rw[3:7] + [''] + rw[7:8] + rw[2:3] + rw[-1:] for rw in data]
        print(data[0], len(data))
    if data is not None:
        cur.executemany('''INSERT INTO {table_name} ('ADDRESS', 'KOR', 'BASE', 'SUF', 'GLOSS',
        'TRANS', 'ContKOR', 'RUS', 'WClass', 'COMMENT') VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''.format(table_name=table_name),
                        data)
    con.commit()

In [ ]:
pr_base = 'data.sqlite'
pr_tables = ['Denis4', 'Dasha5','Nastya6', 'GenConc']
for pr_tab in pr_tables:
    create_table('ver_B', pr_base=pr_base, pr_table=pr_tab)

In [ ]:
pr_files = ['Dasha1.csv', 'Nastya1.tsv', 'Seva1.csv', 'Denis1.csv', 'Dasha2.csv', 'Nastya2.tsv', 'Seva2.csv', 'Denis2.csv', 'Dasha3.csv', 'Nastya3.tsv']
for pr_f in pr_files:
    print(pr_f)
    create_table('ver_C', pr_file=pr_f)

In [2]:
base_name = 'tonmon.sqlite'
con = sqlite3.connect(base_name)
cur = con.cursor()

In [3]:
cur.execute('SELECT DISTINCT GLOSS FROM ver_B')
gs = cur.fetchall()
cur.execute('SELECT DISTINCT GLOSS FROM ver_C')
gs2 = cur.fetchall()
gls = set()
for i in gs + gs2:
    if i[0] and i[0] != '':
        for w in re.split(r'\+|=|-', i[0]):
            gls.add(w.upper())

In [4]:
#gl_dic = {'DECL': ' DECL', 'DECL', 'DECK'}
gl_dic = {}
for g in gls:
    if len(g.strip(' ')) > 4:
        if g.strip(' ')[:4] not in gl_dic:
            gl_dic[g.strip(' ')[:4]] = [g]
        else:
            gl_dic[g.strip(' ')[:4]].append(g)
    else:
        if g not in gl_dic:
            gl_dic[g] = [g]
        else:
            gl_dic[g].append(g)

In [5]:
gl_dic['HONSub'] = ['HONSUBJ', 'HONSUB']
gl_dic['HONSub.VOL'] = ['HONSUB.VOL']
gl_dic['COMIT'] = ['COMI?', 'COMI', 'COMIT']
gl_dic['ADVS'] = ['ADV(S)', 'ADVS', 'ADV']
gl_dic['?'] = ['?', '??', '???']
gl_dic['LOC'] = ['LOC?', 'LOC']
gl_dic['DECL'] = ['DECL/QUOT', 'DECL', 'DECK']
gl_dic['NMNZ'] = ['NOMZ', '?NOMZ', 'NMLZ', 'NMNZ']
gl_dic['TOP'] = ['TOP', '?TOP']
gl_dic['INTER'] = ['INTER', 'INTERWH', 'INTR']
gl_dic['QUOT'] = ['QOUT', 'QUOT']
gl_dic['LYRDECL'] = ['LYCDECL', 'LYRDEC']
gl_dic['NEG'] = ['NEG?', 'NEG']
gl_dic['CV'] = ['CV', 'CONV', 'CONVERB']
gl_dic['SHOULD'] = ['SHOULD']
gl_dic['IND'] = ['IND?', 'IND']
gl_dic['PProb'] = ['PPROB']
gl_dic['DRV{N}'] = ['DRV{N}']

In [6]:
del gl_dic['NMLZ']
del gl_dic['DRV{']
del gl_dic['NOM(']
del gl_dic['SHOU']
del gl_dic['PPRO']
del gl_dic['IND?']
del gl_dic['NEG?']
del gl_dic['HONS']
del gl_dic['COMI']
del gl_dic['ADV(']
del gl_dic['ADV']
del gl_dic['??']
del gl_dic['???']
del gl_dic['LOC?']
del gl_dic['?TOP']
del gl_dic['?NOM']
del gl_dic['DECK']
del gl_dic['NOMZ']
del gl_dic['INTR']
del gl_dic['QOUT']
del gl_dic['LYCD']
del gl_dic['LYRD']
del gl_dic['INTE']

In [7]:
un_gls = {}
for k, vals in gl_dic.items():
    for val in vals:
        un_gls[val] = k

In [8]:
def uni_gloss(un_gls, table_name):
    cur.execute('SELECT ID, GLOSS FROM {}'.format(table_name))
    vals = cur.fetchall()
    for i, gl in vals:
        if gl is not None and gl != '':
            gl = '+'.join([un_gls[s.upper()] for s in re.split('\+|=|-', gl)])
            cur.execute('UPDATE {} SET GLOSS = ? WHERE ID = {}'.format(table_name, i), (gl,))
            con.commit()

In [11]:
table_name = 'ver_B'
uni_gloss(un_gls, 'ver_B')

In [12]:
table_name = 'ver_C'
uni_gloss(un_gls, 'ver_B')